はじめに
-------

Hyにより Chainerのサンプルを記述します。

In [1]:
(require [hy.extra.anaphoric [*]])

Chainerのインストール
-----------------

In [2]:
!pip install chainer

    100% |████████████████████████████████| 348kB 1.2MB/s ta 0:00:011
    100% |████████████████████████████████| 61kB 5.3MB/s eta 0:00:01
    100% |████████████████████████████████| 163kB 3.9MB/s eta 0:00:01
    100% |████████████████████████████████| 102kB 2.2MB/s a 0:00:01
  Running setup.py bdist_wheel for chainer ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/50/a6/5f/1b4d884fcc4e079cfc6f60f1ba0f5d1cffff63be09c111e273
  Running setup.py bdist_wheel for filelock ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/68/e0/0c/ca5c34b3b61ef8d3caf4b7fbbfb2f9f840128ce4015ad52566
Successfully built chainer filelock


Hyで記述した [Chainerのチュートリアル](https://docs.chainer.org/en/stable/tutorial/basic.html#core-concept)を以下に示します。

In [160]:
(import [numpy :as np])
(import chainer)
(import [chainer [cuda Function gradient_check report training utils Variable
                  datasets iterators optimizers serializers Link Chain ChainList]])
(import [chainer.functions :as F]
        [chainer.links :as L])
(import [chainer.training [extensions]])

In [64]:
(def x_data (.astype (np.array 5) np.float32))
(def x (Variable x_data))
(def y (+ (** x 2) (* -2 x) 1))  ;;y = x**2 - 2 * x + 1
(print y.data)

16.0


In [65]:
(.backward y)
(print x.grad)

8.0


In [66]:
(def z (* 2 x))
(def y (+ (** 2 x) (* -1 z) 1))
(.backward y True)
(print z.grad)

-1.0


In [123]:
(def x (Variable (.astype (np.array [[1 2 3] [4 5 6]]) np.float32)))
(def y (+ (** 2 x) (* -2 x) 1))
(def y.grad (.astype (np.ones [2 3]) np.float32))
(.backward y)
(print x.grad)

[[ -0.61370564   0.77258873   3.54517746]
 [  9.09035492  20.18070984  42.36141968]]


In [133]:
(def f (L.Linear 3 2))
(print f.W.data)
(print f.b.data)

[[-0.00143303  0.40721077 -0.38072318]
 [ 0.3487666   0.83152848  0.17531426]]
[ 0.  0.]


In [136]:
(def x (Variable (.astype (np.array [[1 2 3] [4 5 6]]) np.float32)))
(def y (f x))
(print y.data)

[[-0.32918096  2.53776646]
 [-0.25401711  6.60459423]]


In [138]:
(def l1 (L.Linear 4 3))
(def l2 (L.Linear 3 2))
(defn my_forward [x]
  (def h (l1 x))
  (l2 h))

In [143]:
(defclass MyProc [object]
  (defn --init-- [self]
    (def self.l1 (L.Linear 4 3))
    (def self.l2 (L.Linear 3 2)))
  (defn forward [self x]
    (def h (self.l1 x))
    (self.l2 h)))

In [228]:
(defclass MyChain [Chain]
  (defn --init-- [self]
    (.__init__ (super MyChain self))
    (with [(.init_scope self)]
      (def self.l1 (L.Linear 4 3))
      (def self.l2 (L.Linear 3 2))))

  (defn --call-- [self x]
    (def h (self.l1 x))
    (self.l2 h)))

In [227]:
(defclass MyChain2 [ChainList]
  (defn --init-- [self]
    (.__init__ (super MyChain2 self)
                 (L.Linear 4 3)
                 (L.Linear 3 2)))

  (defn --call-- [self x]
    (def h (self[0] x))
    (self[1] h)))

In [233]:
(def model (MyChain))
(def optimizer (.SGD optimizers))
(.setup optimizer model)

In [163]:
(.add_hook optimizer (chainer.optimizer.WeightDecay 0.0005))

In [262]:
(def x (.astype (np.random.uniform -1 1 [2 4]) np.float32))
(.cleargrads model)
;; compute gradient here...
(def loss (.sum F (model (chainer.Variable x))))
(.backward loss)
(.update optimizer)

In [283]:
(defn lossfun [arg1 arg2]
  (def loss (.sum F (model (- arg1 arg2))))
  loss)

(def arg1 (.astype (np.random.uniform -1 1 [2 4]) np.float32))
(def arg2 (.astype (np.random.uniform -1 1 [2 4]) np.float32))
(.update optimizer lossfun (chainer.Variable arg1) (chainer.Variable arg2))

In [291]:
(.save_npz serializers "my.model" model)

In [295]:
(def [train test] (.get_mnist datasets))

In [300]:
(def train_iter (.SerialIterator iterators train (def batch_size 100) (def shuffle True)))

In [301]:
(defclass MLP [Chain]
  (defn --init-- [self n_units n_out]
    (.__init__ super MLP self)
    (with [(.init_scope self)]
      (def self.l1 (L.Linear None n_units))  ;; n_in -> n_units
      (def self.l2 (L.Linear None n_units))  ;; n_units -> n_units
      (def self.l3 (L.Linear None n_out))))  ;; n_units -> n_out

  (defn --call-- [self, x]
    (def h1 (.relu F (self.l1 x)))
    (def h2 (.relu F (self.l2 h1)))
    (def y (self.l3 h2))
    y))

In [302]:
(def model (.Classifier L MLP 100 10))  ;; the input size, 784, is inferred
(def optimizer (.SGD optimizers))
(.setup optimizer model)

In [362]:
(def updater (.StandardUpdater training train_iter optimizer))
(def trainer (.Trainer training updater [20 "epoch"] (def out "result")))

In [363]:
(.run trainer)

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/calysto_hy/kernel.py", line 98, in do_execute_direct
    eval(code, self.env)
  File "In [363]", line 1, in <module>
  File "/opt/conda/lib/python3.6/site-packages/chainer/training/trainer.py", line 271, in run
    os.makedirs(self.out)
  File "/opt/conda/lib/python3.6/os.py", line 205, in makedirs
    head, tail = path.split(name)
  File "/opt/conda/lib/python3.6/posixpath.py", line 105, in split
    p = os.fspath(p)
TypeError: expected str, bytes or os.PathLike object, not NoneType

